In [25]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account

from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import matplotlib.pyplot as plt
from decouple import config
import json
import numpy as np


In [9]:
key_path = '../raw_data/skyscanner-insights-343713-06a72b2724ca.json'
key_path


'../raw_data/skyscanner-insights-343713-06a72b2724ca.json'

In [4]:
credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)


In [5]:
query= '''SELECT
    *
FROM
    `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
WHERE
    Timestamp BETWEEN DATETIME("2022-11-01") AND DATETIME(DATETIME("2023-10-31"))
    AND CabinClass = 'ECONOMY'
    AND OriginCity = 'LON'
    AND DestinationCity = 'JED'
ORDER BY
    Timestamp DESC
'''


In [6]:
query_job = client.query(query)


In [7]:
results = query_job.result()
the_data = results.to_dataframe()


In [7]:
# the_data = the_data.sample(frac=0.1, replace=True, random_state=1)
# the_data.head()


,Guid,Timestamp,Market,Locale,Currency,OriginAirport,OriginCity,OriginCountry,DestinationAirport,DestinationCity,...,Leg_1_segment_3_OriginName,Leg_1_segment_3_DestinationIATA,Leg_1_segment_3_DestinationName,Leg_1_segment_3_DepartureTimestamp,Leg_1_segment_3_ArrivalTimestamp,Leg_1_segment_3_MarketingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_1_segment_3_FlightNumber,Leg_1_segment_3_DurationMin,Leg_1_segment_3_TravelDistanceKm
128037,e978dd8e-095b-11ee-ac45-8e0a9a369c32,2023-06-12 20:01:25,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
267336,f40c3710-882c-11ed-96a6-32b6a667c27b,2022-12-30 10:30:17,UK,en-GB,GBP,STN,LON,GB,JED,JED,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
73349,7b7e4272-4184-11ee-96b7-02ef3633b1b5,2023-08-23 07:12:55,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
117583,0ed605a2-1a14-11ee-b127-0a80c4d41346,2023-07-04 02:39:54,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
21440,a6367b32-6822-11ee-901b-cade9f03e425,2023-10-11 10:40:52,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>


In [9]:
columns = the_data.columns.tolist()
print(columns)


['Guid', 'Timestamp', 'Market', 'Locale', 'Currency', 'OriginAirport', 'OriginCity', 'OriginCountry', 'DestinationAirport', 'DestinationCity', 'DestinationCountry', 'DepartureDate', 'ReturnDate', 'CabinClass', 'Kind', 'TripLengthNights', 'TravelHorizonDays', 'TravelType', 'TravelDistanceKm', 'PassengerAdult', 'PassengerChild', 'PassengerInfant', 'Device', 'BrowserName', 'BrowserVersion', 'OSName', 'OSVersion', 'ProviderType', 'RedirectType', 'ItineraryPosition', 'ItinerarySortType', 'ProviderRank', 'SelfTransfer', 'ProviderIntel', 'SearchLowestPriceRatio', 'ItineraryLowestPriceRatio', 'PriceUserCurrency', 'PriceUserCurrencyPerPax', 'PriceCustomCurrency', 'PriceCustomCurrencyPerPax', 'UserCountryCode', 'UserCountryName', 'UserRegionName', 'UserCityName', 'UserDistrictName', 'UserLongitude', 'UserLatitude', 'PassengerPreference', 'ProviderPreference', 'ThemeAdventureWeight', 'ThemeArtWeight', 'ThemeBeachWeight', 'ThemeCityWeight', 'ThemeFoodWeight', 'ThemeCultureWeight', 'ThemeMountainWe

In [22]:
#create new columns:
# 1. OD_duration
# 2.
def create_columns(the_data):
    #create OD column
    the_data['OD'] = the_data['OriginCity'] + the_data['DestinationCity']

    # Replace N/A or NaN values with 0 for each relevant column
    columns_to_check = [
        'Leg_0_segment_1_DurationMin', 'Leg_0_segment_2_DurationMin', 'Leg_0_segment_3_DurationMin',
        'Leg_1_segment_1_DurationMin', 'Leg_1_segment_2_DurationMin', 'Leg_1_segment_3_DurationMin'
    ]
    for column in columns_to_check:
        the_data[column] = the_data[column].fillna(0)

    # Calculate the OD_duration
    the_data['OD_duration'] = \
        the_data['Leg_0_segment_1_DurationMin']\
        + the_data['Leg_0_segment_2_DurationMin']\
        + the_data['Leg_0_segment_3_DurationMin']\
        + the_data['Leg_1_segment_1_DurationMin']\
        + the_data['Leg_1_segment_2_DurationMin']\
        + the_data['Leg_1_segment_3_DurationMin']
    return the_data


In [11]:
# create a function to score the itinerary on duration and price
def calculate_score(group):
    conditions = [
        group >= group.quantile(0.9),
        group >= group.quantile(0.8),
        group >= group.quantile(0.7),
        group >= group.quantile(0.6),
        group <= group.quantile(0.1),
        group <= group.quantile(0.2),
        group <= group.quantile(0.3),
        group <= group.quantile(0.4),
        group <= group.quantile(0.5),
    ]
    choices = [-5, -4, -3, -2, 5, 4, 3, 2, 1]
    return np.select(conditions, choices, default=0)


In [23]:
create_columns(the_data)


,Guid,Timestamp,Market,Locale,Currency,OriginAirport,OriginCity,OriginCountry,DestinationAirport,DestinationCity,...,Leg_1_segment_3_DestinationName,Leg_1_segment_3_DepartureTimestamp,Leg_1_segment_3_ArrivalTimestamp,Leg_1_segment_3_MarketingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_1_segment_3_FlightNumber,Leg_1_segment_3_DurationMin,Leg_1_segment_3_TravelDistanceKm,OD_duration,OD
128037,e978dd8e-095b-11ee-ac45-8e0a9a369c32,2023-06-12 20:01:25,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,185,LONJED
267336,f40c3710-882c-11ed-96a6-32b6a667c27b,2022-12-30 10:30:17,UK,en-GB,GBP,STN,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,435,LONJED
73349,7b7e4272-4184-11ee-96b7-02ef3633b1b5,2023-08-23 07:12:55,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,155,LONJED
117583,0ed605a2-1a14-11ee-b127-0a80c4d41346,2023-07-04 02:39:54,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,0,LONJED
21440,a6367b32-6822-11ee-901b-cade9f03e425,2023-10-11 10:40:52,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,625,LONJED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47456,c47d334a-5622-11ee-92fd-d64cb0e38b60,2023-09-18 12:56:22,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,285,LONJED
233133,c1438bb0-9e9a-11ed-8405-ee5010959783,2023-01-27 23:31:42,UK,en-GB,GBP,LGW,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,670,LONJED
240559,142bc972-995b-11ed-8799-de254060e4b0,2023-01-21 07:13:17,UK,en-GB,GBP,LHR,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,610,LONJED
235124,a38696ae-9d04-11ed-ab47-ee5010959783,2023-01-25 23:04:36,UK,en-US,GBP,STN,LON,GB,JED,JED,...,,NaT,NaT,,,<NA>,0,<NA>,415,LONJED


In [26]:
the_data['OD_duration_score'] = the_data.groupby('OD')['OD_duration'].transform(calculate_score)


TypeError: invalid entry 0 in condlist: should be boolean ndarray

In [31]:
data_subset = the_data[colums_subset].copy()
data_subset


,Market,OriginAirport,DestinationAirport,Locale,CabinClass,Kind,TripLengthNights,TravelType,TravelDistanceKm,TravelHorizonDays,...,Leg_0_segment_0_OperatingCarrierIATA,Leg_0_segment_1_OperatingCarrierIATA,Leg_0_segment_2_OperatingCarrierIATA,Leg_0_segment_3_OperatingCarrierIATA,Leg_1_segment_0_OperatingCarrierIATA,Leg_1_segment_1_OperatingCarrierIATA,Leg_1_segment_2_OperatingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_0_DurationMin,Leg_1_DurationMin
0,ES,STN,AAR,es-ES,ECONOMY,RETURN,9,INTERNATIONAL,836,37,...,FR,,,,FR,,,,100,105
1,NG,PHC,ABV,en-GB,ECONOMY,RETURN,7,DOMESTIC,444,5,...,P4,,,,P4,,,,70,70
2,MX,MEX,ACA,es-MX,ECONOMY,RETURN,2,DOMESTIC,307,8,...,Y4,,,,Y4,,,,62,83
3,UK,LGW,ACC,en-GB,ECONOMY,RETURN,9,INTERNATIONAL,5065,75,...,BA,,,,BA,,,,395,400
4,UK,LHR,ACC,en-GB,ECONOMY,RETURN,21,INTERNATIONAL,5101,37,...,BA,,,,BA,,,,400,410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,NL,SFO,AMS,nl-NL,ECONOMY,RETURN,31,INTERNATIONAL,8786,60,...,KL,,,,KL,,,,615,660
196,KR,ICN,AMS,ko-KR,ECONOMY,RETURN,6,INTERNATIONAL,8552,37,...,KL,,,,KE,,,,810,715
197,NL,ICN,AMS,en-GB,ECONOMY,RETURN,89,INTERNATIONAL,8552,14,...,KE,,,,KE,,,,810,640
198,PL,KRK,AOI,pl-PL,ECONOMY,RETURN,4,INTERNATIONAL,868,26,...,RR,,,,RR,,,,100,100


In [32]:
data_subset['OD'] = data_subset['OriginAirport'] + data_subset['DestinationAirport']


In [33]:
data_subset[data_subset['CabinClass'] == 'ECONOMY']


,Market,OriginAirport,DestinationAirport,Locale,CabinClass,Kind,TripLengthNights,TravelType,TravelDistanceKm,TravelHorizonDays,...,Leg_0_segment_1_OperatingCarrierIATA,Leg_0_segment_2_OperatingCarrierIATA,Leg_0_segment_3_OperatingCarrierIATA,Leg_1_segment_0_OperatingCarrierIATA,Leg_1_segment_1_OperatingCarrierIATA,Leg_1_segment_2_OperatingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_0_DurationMin,Leg_1_DurationMin,OD
0,ES,STN,AAR,es-ES,ECONOMY,RETURN,9,INTERNATIONAL,836,37,...,,,,FR,,,,100,105,STNAAR
1,NG,PHC,ABV,en-GB,ECONOMY,RETURN,7,DOMESTIC,444,5,...,,,,P4,,,,70,70,PHCABV
2,MX,MEX,ACA,es-MX,ECONOMY,RETURN,2,DOMESTIC,307,8,...,,,,Y4,,,,62,83,MEXACA
3,UK,LGW,ACC,en-GB,ECONOMY,RETURN,9,INTERNATIONAL,5065,75,...,,,,BA,,,,395,400,LGWACC
4,UK,LHR,ACC,en-GB,ECONOMY,RETURN,21,INTERNATIONAL,5101,37,...,,,,BA,,,,400,410,LHRACC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,NL,SFO,AMS,nl-NL,ECONOMY,RETURN,31,INTERNATIONAL,8786,60,...,,,,KL,,,,615,660,SFOAMS
196,KR,ICN,AMS,ko-KR,ECONOMY,RETURN,6,INTERNATIONAL,8552,37,...,,,,KE,,,,810,715,ICNAMS
197,NL,ICN,AMS,en-GB,ECONOMY,RETURN,89,INTERNATIONAL,8552,14,...,,,,KE,,,,810,640,ICNAMS
198,PL,KRK,AOI,pl-PL,ECONOMY,RETURN,4,INTERNATIONAL,868,26,...,,,,RR,,,,100,100,KRKAOI


In [34]:
data_subset['Leg_0_segment_0_OperatingCarrierIATA'].value_counts(normalize=True)[:30]


Leg_0_segment_0_OperatingCarrierIATA
FR    0.195
U2    0.150
KL    0.080
VY    0.070
      0.045
LS    0.025
XQ    0.020
RJ    0.020
PC    0.020
EI    0.020
EW    0.015
WA    0.015
RR    0.015
BA    0.015
5O    0.015
HV    0.015
W9    0.015
J9    0.010
QP    0.010
CX    0.010
W6    0.010
BY    0.010
AT    0.010
V7    0.010
I2    0.010
AR    0.010
SK    0.010
4C    0.005
FZ    0.005
FO    0.005
Name: proportion, dtype: float64